In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [2]:
# Standardization formula: z = (x - mean) / standard_deviation

class Standardiser():
    def __init__(self) -> None:
        pass

    def fit(self, X):
        self.size = X.shape[1]
        self.mean = X.mean(axis=0)
        self.std = X.std(axis=0)
        return self     #returns self, which allows method chaining

    def transform(self,X):
        if X.shape[1] != self.size:
            raise Exception("Wrong array dimensions!")
        return (X-self.mean)/self.std


In [3]:
def shuffle_data(X, Y):
    data_num = np.arange(X.shape[0])
    np.random.shuffle(data_num)
    return X[data_num], Y[data_num]

def train_test_split(X, Y, test_size=0.5, shuffle=False):
    """
    Splits dataset into training and test sets.

    Parameters:
    ----------
    X : array-like
        Feature dataset.
    y : array-like
        Target values.
    test_size : float or int, default=0.5
        - If float (0 < test_size < 1), it represents the proportion of the dataset to include in the test split.
        - If int (1 <= test_size < len(y)), it represents the absolute number of test samples.
    shuffle : bool, default=True
        If True, shuffles data before splitting.

    Returns:
    -------
    X_train, X_test, y_train, y_test : array-like
        Split feature and target sets for training and testing."""

    if shuffle:
        X, Y = shuffle_data(X, Y)
    if test_size <1 :
        train_ratio = len(Y) - int(len(Y) *test_size)
        X_train, X_test = X[:train_ratio], X[train_ratio:]
        Y_train, Y_test = Y[:train_ratio], Y[train_ratio:]
        return X_train, X_test, Y_train, Y_test
    elif test_size in range(1,len(Y)):
        X_train, X_test = X[test_size:], X[:test_size]
        Y_train, Y_test = Y[test_size:], Y[:test_size]
        return X_train, X_test, Y_train, Y_test 

In [4]:
def evaluate_model(Y_true, Y_pred, metric="rmse", normalize=True):
    """
    Evaluate the model based on the specified metric.

    Parameters:
    - Y_true (array-like): True labels.
    - Y_pred (array-like): Predicted labels.
    - metric (str): The metric to use for evaluation. Options are "class_accuracy" and "rmse".
    - normalize (bool): If True, returns accuracy as a fraction; otherwise, returns the count.

    Returns:
    - float: Calculated score based on the specified metric.
    """
    
    Y_true, Y_pred = np.ravel(Y_true), np.ravel(Y_pred)  # Flatten to 1D

    if Y_true.shape != Y_pred.shape:
        raise ValueError("Y_true and Y_pred must have the same number of elements.")
    
    if metric == "class_accuracy":
        score = np.mean(Y_true == Y_pred) if normalize else np.sum(Y_true == Y_pred)
    elif metric == "rmse":
        score = np.sqrt((np.square(Y_pred - Y_true)).mean())
    else:
        raise ValueError("Invalid metric. Choose either 'class_accuracy' or 'rmse'.")
    return score

The logistic regression algorithm is implemented within the **LogisticRegression()** class. This class can perform both binary and multi-class classification for input data of arbitrary size. In multi-class classification, decisions are made using the one_vs_all approach. Mini-batch gradient descent algorithm is used, where the mb_size parameter indicates the percentage of the training set utilized in each iteration. The theta and cost fields store parameters for all classifiers and the corresponding cost function values during training.

In [5]:
class LogisticRegression():
    def __init__(self, alpha, num_iter, mb_size=0.2, rs=None):
        self.alpha = alpha
        self.num_iter = num_iter
        self.mb = mb_size
        np.random.seed(rs)

    # Sigmoid function
    def _sigmoid_function(self, X):
        return 1 / (1 + np.exp(-X))

    # Compute hypothesis function (prediction) using sigmoid on linear combination of inputs
    def _h_theta(self, theta, X):
        return self._sigmoid_function(X @ theta)

    # Compute the logistic regression cost function for current predictions
    def _cost_function(self, X, Y, theta):
        h = self._h_theta(theta, X)
        # Clip values of h to avoid log(0) errors
        h = np.clip(h, 1e-10, 1 - 1e-10)
        # Return the average cost for binary classification
        return -np.mean(Y * np.log(h) + (1 - Y) * np.log(1 - h))

    # Calculate the gradient of the cost function for parameter update
    def _calc_gradient(self, X, Y, theta):
        return (X.T @ (Y - self._h_theta(theta, X))) / len(Y)

    def fit(self, X, Y):
        self.dim = X.ndim
        self.classes = np.unique(Y)  # Unique class labels for multi-class handling

        # Add a column of ones to X for the bias (intercept) term
        X = self._add_ones(X)

        # Initialize theta and cost matrices to store parameters and cost values for each class
        self.theta = np.zeros((len(self.classes), X.shape[1]))
        self.cost = np.zeros((len(self.classes), self.num_iter))

        # Determine minibatch size as a fraction of total data length
        l = len(X)
        self.mb_size = int(self.mb * l)

        # Train the model for each class (for multi-class logistic regression)
        for j, c in enumerate(self.classes):
            y_temp = np.where(Y == c, 1, 0)  # Convert labels to binary for the current class
            # Perform gradient descent for specified number of iterations
            for i in range(self.num_iter):
                # Extract a random minibatch of data
                ind = np.random.permutation(l)[:self.mb_size]
                X_mb = X[ind, :]
                Y_mb = y_temp[ind]

                # Update theta using the gradient descent step
                gradient = self._calc_gradient(X_mb, Y_mb, self.theta[j])
                self.theta[j] += self.alpha * gradient

                # Calculate and store the cost for the current minibatch
                self.cost[j, i] = self._cost_function(X_mb, Y_mb, self.theta[j])
        return self

    def predict(self, X):
        if X.ndim != self.dim:
            X = np.array([X])
        X = self._add_ones(X)
        # Predict the class label with the highest probability
        out = [self.classes[int(i)] for i in np.argmax(self._h_theta(self.theta.T, X), axis=1)]
        return np.array(out)

    def _add_ones(self, X):
        if type(X) != np.ndarray:
            X = np.array(X)
        return np.concatenate((np.ones((len(X), 1)), X), axis=1)


Program running:

In [6]:
#Read the data
dile_name = 'data-class.csv'
raw_data = pd.read_csv(dile_name, header=None).to_numpy()
X = raw_data[:,:5]
Y = raw_data[:,-1]

#Split data to train test datasets
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,0.2,shuffle=True)

#Standardisation of the data 
scaler = Standardiser().fit(X_train)
try:
    X_train_scaled = scaler.transform(X_train)
    X_test_scaled = scaler.transform(X_test)
except Exception as e:
  print(e)

The first model (one model has 3 classifiers in it) is trained using the "optimal" parameters for α and mb_size. In addition to this "optimal" configuration, four other models were trained, each combining one optimal parameter with another that is either larger or smaller than the "optimal" value. The resulting graph illustrates the relationship between the negative log-likelihood and the number of samples used for training. To enhance clarity, only the log-likelihood values of one of three classifiers per model are displayed on the graph.

In [7]:
# Define optimal parameters
alpha_opt = 0.15
mb_opt = 0.5
parameters = [
    (alpha_opt, 1),             # Big mini-batch (the whole traning is used as a batch)
    (alpha_opt, mb_opt / 8),    # Small mini-batch
    (alpha_opt * 1000, mb_opt),  # Large learning rate
    (alpha_opt / 1000, mb_opt),  # Small learning rate
    (alpha_opt, mb_opt)         # "Optimal" parameters
]

# Set up the plot
plt.figure(figsize=(10, 7), dpi=1000)

# Iterate over each parameter combination and plot results
for alpha, mb_size in parameters:
    # Train the model with the given parameters
    lr = LogisticRegression(alpha=alpha, num_iter=int(100 * mb_opt / mb_size), mb_size=mb_size, rs=3)
    lr.fit(X_train_scaled, Y_train)
    
    # Plot the cost over the number of samples
    num_samples = np.arange(len(lr.cost[1, :])) * lr.mb_size
    plt.plot(num_samples, lr.cost[1, :], label=r"$\alpha={:.3f}$, mb_size={:.3f}".format(alpha,mb_size))

# Customize plot appearance
plt.legend(loc='upper right')
plt.xlim(0, 5000)
plt.ylim(-0.05, 1)
plt.xlabel('Number of Samples')
plt.ylabel(r'- l ($\dot{\Theta}$)')
plt.title('Gradient Descent')
plt.grid()

# Adjust layout and display
plt.tight_layout()
plt.show()

The blue line (mini-batch size equal to the entire training set) exhibits a gradual decrease in the negative log-likelihood. However, the convergence is relatively slow because the model must process the entire dataset for each update. Despite the slower rate of convergence, the training set size is small, which allows the overall process to remain relatively efficient.

The orange line (small mini-batch) exhibits significant fluctuations, indicating instability in convergence, due to a very small mini-batch size.

The green line (with a high learning rate) shows a poor convergence trend, potentially diverging due to the excessively high learning rate.

The red line (with a very low learning rate) shas very slow convergence, taking a long time to decrease the cost.

The purple line displays a stable yet slower convergence rate, indicating a more balanced choice of parameters.

Optimal model performance:

In [8]:
# alpha_opt = 0.4
# mb_opt = 0.2
# lr = LogisticRegression(alpha=alpha, num_iter=int(1000 * mb_opt / mb_size), mb_size=mb_size, rs=3)
# lr.fit(X_train_scaled, Y_train)

try:
    # Training set performance
    Y_pred_train = lr.predict(X_train_scaled)
    accuracy_train =  evaluate_model(Y_train, Y_pred_train, metric="class_accuracy")
    print(f"Training set accuracy: {accuracy_train * 100:.2f}%")
except Exception as e:
    print(e)

# Test set performance
try:
    Y_pred_test = lr.predict(X_test_scaled)
    accuracy_test =  evaluate_model(Y_pred_test, Y_test, metric="class_accuracy")
    print(f"Test set ccuracy: {accuracy_test * 100:.2f}%")
except Exception as e:
    print(e)

Training set accuracy: 97.20%
Test set ccuracy: 94.29%


---
# Built-in functions results:

In [9]:
# Import necessary libraries
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler

# Split the dataset into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

# Initialize and train the logistic regression model
model = LogisticRegression(max_iter=200)  # Setting max_iter to ensure convergence
model.fit(X_train, Y_train)

# Make predictions on the test data
Y_pred_train = model.predict(X_train)
Y_pred_test = model.predict(X_test)

# Evaluate the model
accuracy_train = accuracy_score(Y_train, Y_pred_train)
accuracy_test = accuracy_score(Y_test, Y_pred_test)
report = classification_report(Y_test, Y_pred_test)

print(f"Built-in model accuracy on train set: {accuracy_train * 100:.2f}%")
print(f"Built-in model accuracy on test set: {accuracy_test * 100:.2f}%")
print("\nClassification Report:\n", report)


Built-in model accuracy on train set: 98.39%
Built-in model accuracy on test set: 98.15%

Classification Report:
               precision    recall  f1-score   support

         0.0       0.95      1.00      0.97        19
         1.0       1.00      0.96      0.98        24
         2.0       1.00      1.00      1.00        11

    accuracy                           0.98        54
   macro avg       0.98      0.99      0.98        54
weighted avg       0.98      0.98      0.98        54

